<a href="https://colab.research.google.com/github/Yash-Yelave/Recomendation_systems/blob/main/RS_PR2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Sample data
data =

# {
#     'user': ['A', 'A', 'B', 'B', 'C', 'C'],
#     'item': ['Item1', 'Item2', 'Item1', 'Item3', 'Item2', 'Item3'],
#     'rating': [5, 3, 4, 2, 4, 5]
# }

df = pd.DataFrame(data)

# Create user-item matrix
user_item_matrix = df.pivot_table(index='user', columns='item', values='rating')


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Fill NaNs with 0 for similarity calculation
user_similarity = cosine_similarity(user_item_matrix.fillna(0))

# Convert to DataFrame for easier interpretation
similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)


In [ ]:
def get_top_k_similar_users(user_id, k=2):
    similar_users = similarity_df[user_id].drop(user_id)
    return similar_users.sort_values(ascending=False).head(k)


In [ ]:
def predict_ratings(user_id, k=2):
    similar_users = get_top_k_similar_users(user_id, k)
    user_ratings = user_item_matrix.loc[user_id]

    # Items not yet rated by user
    unseen_items = user_ratings[user_ratings.isna()].index

    predicted_ratings = {}

    for item in unseen_items:
        numerator = 0
        denominator = 0
        for sim_user, sim_score in similar_users.items():
            if pd.notna(user_item_matrix.loc[sim_user, item]):
                numerator += sim_score * user_item_matrix.loc[sim_user, item]
                denominator += sim_score

        if denominator != 0:
            predicted_ratings[item] = numerator / denominator

    return predicted_ratings


In [ ]:
recommendations = predict_ratings('A', k=2)
sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)

print("Recommendations for User A:", sorted_recommendations)


In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

# Load your data
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user', 'item', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2)

# User-based CF
sim_options = {
    'name': 'cosine',
    'user_based': True
}

algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

# Predict
predictions = algo.test(testset)
